# ABS Monthly Household Spending Indicator 5682

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_ABS_meta_and_data,
    get_plot_constants,
    find_rows,
    get_meta_constants,
)
from plotting import (
    finalise_plot,
    set_chart_dir,
    clear_chart_dir,
    plot_growth_finalise,
    line_plot,
    calc_growth,
    COLOR_BLUE,
    COLOR_AMBER,
    COLOR_RED,
)
from henderson import hma

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
catalogue_id = "5682"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_dict = get_ABS_meta_and_data(catalogue_id, verbose=False)
if abs_dict is None:
    sys.exit(-1)
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Retrieving zip-file from cache ...
Extracting DataFrames from the zip-file ...


## Plot

### Set-up 

In [3]:
plt.style.use("fivethirtyeight")
set_chart_dir(CHART_DIR)
clear_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Headline growth - calendar adjusted

In [4]:
table = '01'
adjustment = 'Calendar adjusted'
prices = 'Current Price'
base_selector = {
    table: table_col,
    'household spending': did_col,
    adjustment: did_col,
    prices: did_col,
}
annual_selector = base_selector | {
    'Through the year percentage change': did_col,
}
annual_rows = find_rows(meta, annual_selector, verbose=True)
month_selector = base_selector | {
    'Monthly percentage change': did_col,
}
month_rows = find_rows(meta, month_selector, verbose=True)

Searching 756: term: 01 in-column: Table
Searching 84: term: household spending in-column: Data Item Description
Searching 42: term: Calendar adjusted in-column: Data Item Description
Searching 42: term: Current Price in-column: Data Item Description
Searching 42: term: Through the year percentage change in-column: Data Item Description
14
Searching 756: term: 01 in-column: Table
Searching 84: term: household spending in-column: Data Item Description
Searching 42: term: Calendar adjusted in-column: Data Item Description
Searching 42: term: Current Price in-column: Data Item Description
Searching 42: term: Monthly percentage change in-column: Data Item Description
14


In [5]:
LONG_TITLE = 20
data = abs_dict[table]
for chart in [x.split(';')[1].strip() for x in annual_rows[did_col]]:
    
    # data wrangling
    annual_id = annual_rows[annual_rows[did_col].str.contains(chart, regex=False)][id_col].iloc[0]
    month_id =  month_rows[month_rows[did_col].str.contains(chart, regex=False)][id_col].iloc[0]
    annual, month = data[annual_id], data[month_id]
    
    # plotting
    separator = ' ' if len(chart) <= LONG_TITLE else '\n'
    plot_growth_finalise(
        annual,
        month,
        title=f'Growth in Household Spending:{separator}{chart}',
        ylabel='Per cent',
        lfooter=f'Australia. {prices}. {adjustment}. ',
        rfooter=source,
        show=SHOW,
    )

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sun Apr 09 2023 12:53:59

Python implementation: CPython
Python version       : 3.11.3
IPython version      : 8.12.0

matplotlib: 3.7.1
sys       : 3.11.3 | packaged by conda-forge | (main, Apr  6 2023, 09:05:00) [Clang 14.0.6 ]
pandas    : 2.0.0

Watermark: 2.3.1



In [7]:
print("Done")

Done
